In [1]:
import numpy as np
from PIL import Image
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Mounted at /content/drive


Всего у нас  1070 чб изображений, форматы могут как jpg, или img, так и другие. Каждое из них является капчей, состоящей из 5 символов. Названия картинок отображают содержание капчи. Например 2ap5g.jpg. Размеры каждого изображения - 50 на 200 пикселей. Для начала проверим, действительно ли используются все числа и буквы английского алфавита.

In [2]:
dir='/content/drive/My Drive/samples/samples'
chars = {}
for captcha in  os.listdir(dir):
  for i in captcha.split(".")[0]:
    chars[i] = chars.get(i, 0) +1
print(len(chars))
print(*chars.keys())

19
d 7 n 3 4 2 w 6 f 5 x p c y m g e b 8


Видим, что используется всего 19 символов, значит будем тренировать нашу модель всего на 19 символах, что позволит нам существенно улучшить результат. Подготовим наши данные. Будем сопоставлять в y не саму расшифрованную капчу, а матрицу размера 5 (длина любой капчи) на 19 (количество символов). Так у нас для каждой картинки будет матрица, в которой будет по 1 единице в строчке - в столбце, в котором стоит символ

In [3]:
X = np.zeros((1070, 50, 200, 1))  # Добавляем дополнительное измерение для канала
y = np.zeros((5, 1070, 19))
symbols = list(chars.keys())
print(symbols)
for i, captcha in enumerate(os.listdir(dir)):
    captcha_without_format = captcha.split(".")[0]
    captcha_image = cv2.imread(os.path.join(dir, captcha), cv2.IMREAD_GRAYSCALE) / 255.0 #нормируем
    captcha_image = np.expand_dims(captcha_image, axis=-1)  # Добавляем дополнительное измерение
    matr = np.zeros((5, 19))
    X[i] = captcha_image
    for place, symbol in enumerate(captcha_without_format):
        matr[place, list(chars.keys()).index(symbol)] = 1
    y[:, i] = matr

print(X.shape)

['d', '7', 'n', '3', '4', '2', 'w', '6', 'f', '5', 'x', 'p', 'c', 'y', 'm', 'g', 'e', 'b', '8']
(1070, 50, 200, 1)


Разделим данные на тренировочные, тестовые и валидационные вручную, потому что мне не охота возиться. Будем использовать следующие пропорции: 0.7 на тренировочные, 0.15 на тестовые и 0.15 на валидационные данные.

In [4]:
train_indices = np.arange(X.shape[0])
np.random.shuffle(train_indices)
X= X[train_indices]
y = y[:, train_indices]

X_train = X[: 749]
y_train = y[:, :749]
X_val = X[749 : 910]
y_val = y[:, 749 : 910]
X_test = X[910:]
y_test = y[:, 910:]

y_train = np.transpose(y_train, (1, 0, 2))
y_test = np.transpose(y_test, (1, 0, 2))
y_val = np.transpose(y_val, (1, 0, 2))

# Проверка формы разделенных данных
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (749, 50, 200, 1)
y_train shape: (749, 5, 19)
X_test shape: (160, 50, 200, 1)
y_test shape: (160, 5, 19)
X_val shape: (161, 50, 200, 1)
y_val shape: (161, 5, 19)


Я собираюсь использовать сверточную нейронную сеть для решения задачи, ведь они как раз подходят для работы с изображениями.

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import Reshape
import keras

In [6]:
def build_model_4():
    # Входной слой
    inp_layer = Input(shape=(50, 200, 1))  # Входные данные имеют форму (50, 200, 1)

    # Первый сверточный слой
    convol_layer_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(inp_layer)  # 32 фильтра размером 3x3
    pool_1 = MaxPooling2D((2, 2), padding='same')(convol_layer_1)

    # Второй сверточный слой
    convol_layer_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(pool_1)
    pool_2 = MaxPooling2D((2, 2), padding='same')(convol_layer_2)

    # третий сверточный слой
    convol_layer_3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool_2)
    pool_3 = MaxPooling2D((2, 2), padding='same')(convol_layer_3)

    fl = Flatten()(pool_3)
    x = Dropout(0.4)(fl)
    # пытаемся предотвратить переобучение
    x = BatchNormalization()(x)
    outputs = []


    # для каждой отдельной буквы создаем отдельную как бы ветку, чтобы можно было угадывать ее отдельно
    dense1 = Dense(64 , activation='relu')(x)
    dropout1= Dropout(0.5)(dense1)
    dpo1 = BatchNormalization()(dropout1)
    output1 = Dense(19 , activation='softmax')(dpo1)
    outputs.append(output1)

    dense2 = Dense(64 , activation='relu')(x)
    dropout2= Dropout(0.5)(dense2)
    dpo2 = BatchNormalization()(dropout2)
    output2 = Dense(19 , activation='softmax')(dpo2)
    outputs.append(output2)

    dense3 = Dense(64 , activation='relu')(x)
    dropout3= Dropout(0.5)(dense3)
    dpo3 = BatchNormalization()(dropout3)
    output3 = Dense(19 , activation='softmax')(dpo3)
    outputs.append(output3)

    dense4 = Dense(64 , activation='relu')(x)
    dropout4= Dropout(0.5)(dense4)
    dpo4 = BatchNormalization()(dropout4)
    output4 = Dense(19 , activation='softmax')(dpo4)
    outputs.append(output4)

    dense5 = Dense(64 , activation='relu')(x)
    dropout5= Dropout(0.5)(dense5)
    dpo5 = BatchNormalization()(dropout5)
    output5 = Dense(19 , activation='softmax')(dpo5)
    outputs.append(output5)

    model = Model(inputs=inp_layer, outputs=outputs)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss="categorical_crossentropy", metrics=["accuracy"] * 5)

    return model

model_4 = build_model_4()
model_4.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 50, 200, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 50, 200, 32)    │            320 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 25, 100, 32)    │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 25, 100, 64)    │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 13, 50, 64)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 13, 50, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 7, 25, 128)     │              0 │ conv2d_2[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 22400)          │              0 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 22400)          │              0 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 22400)          │         89,600 │ dropout[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │      1,433,664 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │      1,433,664 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │      1,433,664 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │      1,433,664 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 64)             │      1,433,664 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64)             │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)  

 Total params: 7,358,047 (28.07 MB)

 Trainable params: 7,312,607 (27.90 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [7]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Метрика для мониторинга
    patience=10,         # Количество эпох без улучшения, после которых остановить обучение
    restore_best_weights=True  # Восстановить лучшие веса после окончания обучения
)


history = model_4.fit(
    X_train, [y_train[:, i] for i in range(5)],
    batch_size=32,
    epochs=100,
    validation_data=(X_val, [y_val[:, i] for i in range(5)]),
    callbacks=[early_stopping]
)


Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 28s 516ms/step - dense_1_accuracy: 0.0692 - dense_3_accuracy: 0.0756 - dense_5_accuracy: 0.0546 - dense_7_accuracy: 0.0735 - dense_9_accuracy: 0.0379 - loss: 17.4217 - val_dense_1_accuracy: 0.1801 - val_dense_3_accuracy: 0.1739 - val_dense_5_accuracy: 0.1242 - val_dense_7_accuracy: 0.0559 - val_dense_9_accuracy: 0.0807 - val_loss: 14.6057
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - dense_1_accuracy: 0.2402 - dense_3_accuracy: 0.1912 - dense_5_accuracy: 0.1445 - dense_7_accuracy: 0.1418 - dense_9_accuracy: 0.1291 - loss: 14.1617 - val_dense_1_accuracy: 0.3602 - val_dense_3_accuracy: 0.1988 - val_dense_5_accuracy: 0.0621 - val_dense_7_accuracy: 0.2236 - val_dense_9_accuracy: 0.1242 - val_loss: 14.2831
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - dense_1_accuracy: 0.5138 - dense_3_accuracy: 0.4405 - dense_5_accuracy: 0.3811 - dense_7_accuracy: 0.3763 - dense_9_accuracy: 0.3247 - loss: 10.1210 - val_dense_1_accuracy: 0.4099 - val_d

In [8]:
def makePredict(captcha):
    captcha = np.reshape(captcha , (1, 50,200,1))
    result = model_4.predict(captcha)
    result = np.reshape(result ,(5,19))

    label = ''.join([symbols[np.argmax(i)] for i in result])

    return label
def capcha_back(y, symbols = symbols):
  label = ['','','','','']
  for i in range(5):
    for j in range(19):
      if y[i][j] == 1:
        label[i] = symbols[j]
  return ''.join(label)

print(symbols)
count = 0
for i in range(X_test.shape[0]):
  pred = makePredict(X_test[i])
  if pred != capcha_back(y_test[i]):
    count+=1




['d', '7', 'n', '3', '4', '2', 'w', '6', 'f', '5', 'x', 'p', 'c', 'y', 'm', 'g', 'e', 'b', '8']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 624ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [9]:
print(X_test.shape[0], count, ((X_test.shape[0] - count) / X_test.shape[0] * 100))

160 23 85.625


Таким образом точность нашей модели на тестовой выборке составила 85.625 процентов, мне кажется точность могла улучшиться, если бы мы распологали большим количеством данных, хотя-бы в 2 раза.